# Investigation: Coil KMH1 Unloading - 9 rows vs 3 trucks

Checking why Loading Durations Status shows 9 Coil KMH1 Unloading entries but Daily Performance shows only 3 trucks

In [10]:
import pandas as pd
from config.config import SPREADSHEET_ID, SHEET_GIDS
from data.processor import clean_sheet_dfs
from data.metrics import compute_per_truck_metrics

# Load and clean data
def _sheet_csv_url(gid: str):
    return f"https://docs.google.com/spreadsheets/d/{SPREADSHEET_ID}/export?format=csv&gid={gid}"

raw_dfs = {}
for sheet_name in ['security', 'driver', 'status', 'logistic']:
    url = _sheet_csv_url(SHEET_GIDS[sheet_name])
    raw_dfs[sheet_name] = pd.read_csv(url, dtype=str, keep_default_na=False)

dfs = clean_sheet_dfs(raw_dfs)

# Get today's date
today = pd.Timestamp.now(tz="Asia/Phnom_Penh").date()
print(f"Analyzing data for: {today}\n")

# Compute KPI for Coil KMH1
df_kpi = compute_per_truck_metrics(
    dfs['security'], dfs['status'], dfs['logistic'], dfs['driver'],
    start_date=today,
    end_date=today,
    product_filter=["Coil KMH1"],
    upload_type="Unloading",
    use_fallbacks=False
)

print(f"=== KPI DATA FOR COIL KMH1 UNLOADING ===")
print(f"Total KPI rows: {len(df_kpi)}")
print(f"\nUnique trucks: {df_kpi['Truck_Plate_Number'].nunique()}")
print(f"Truck list: {sorted(df_kpi['Truck_Plate_Number'].unique())}")

# Show all rows
print("\n=== ALL KPI ROWS ===")
print(df_kpi[["Truck_Plate_Number", "Product_Group", "Date"]].to_string(index=False))

Analyzing data for: 2025-12-23

=== KPI DATA FOR COIL KMH1 UNLOADING ===
Total KPI rows: 9

Unique trucks: 9
Truck list: ['3A-2389', '3A-2389-2', '3A-2389-3', '3A-6126', '3A-6355', '3A-6355-3', '3B-5677', '3B-5677-2', 'TK-0100']

=== ALL KPI ROWS ===
Truck_Plate_Number Product_Group       Date
           3A-2389     Coil KMH1 2025-12-23
         3A-2389-2     Coil KMH1 2025-12-23
         3A-2389-3     Coil KMH1 2025-12-23
           3A-6126     Coil KMH1 2025-12-23
           3A-6355     Coil KMH1 2025-12-23
         3A-6355-3     Coil KMH1 2025-12-23
           3B-5677     Coil KMH1 2025-12-23
         3B-5677-2     Coil KMH1 2025-12-23
           TK-0100     Coil KMH1 2025-12-23


## Check Status Records - Are there multiple product groups per truck?

In [11]:
# Check if trucks have multiple product groups in status records
df_status = dfs['status'].copy()
df_status["Timestamp"] = pd.to_datetime(df_status["Timestamp"], errors="coerce")
df_status["Date"] = df_status["Timestamp"].dt.date

status_today = df_status[df_status["Date"] == today]

# Get Coil KMH1 trucks from KPI
coil_trucks = df_kpi["Truck_Plate_Number"].unique()

print("=== STATUS RECORDS CHECK ===")
print(f"Checking {len(coil_trucks)} trucks\n")

for truck in sorted(coil_trucks):
    truck_status = status_today[status_today["Truck_Plate_Number"] == truck]
    if not truck_status.empty:
        products = truck_status["Product_Group"].unique()
        print(f"\n{truck}:")
        print(f"  Product Groups: {list(products)}")
        print(f"  Status events: {len(truck_status)}")
        
        # Group by product to see events per product
        for product in products:
            prod_events = truck_status[truck_status["Product_Group"] == product]
            statuses = prod_events["Status"].tolist()
            print(f"    {product}: {statuses}")

=== STATUS RECORDS CHECK ===
Checking 9 trucks


3A-2389:
  Product Groups: ['Coil KMH1']
  Status events: 2
    Coil KMH1: ['Start_Loading', 'Completed']

3A-2389-2:
  Product Groups: ['Coil KMH1']
  Status events: 2
    Coil KMH1: ['Start_Loading', 'Completed']

3A-2389-3:
  Product Groups: ['Coil KMH1']
  Status events: 2
    Coil KMH1: ['Start_Loading', 'Completed']

3A-6126:
  Product Groups: ['Coil KMH1']
  Status events: 2
    Coil KMH1: ['Start_Loading', 'Completed']

3A-6355:
  Product Groups: ['Coil KMH1']
  Status events: 2
    Coil KMH1: ['Start_Loading', 'Completed']

3A-6355-3:
  Product Groups: ['Coil KMH1']
  Status events: 2
    Coil KMH1: ['Start_Loading', 'Completed']

3B-5677:
  Product Groups: ['Coil KMH1']
  Status events: 3
    Coil KMH1: ['Start_Loading', 'Completed', 'Start_Loading']

3B-5677-2:
  Product Groups: ['Coil KMH1']
  Status events: 2
    Coil KMH1: ['Start_Loading', 'Completed']

TK-0100:
  Product Groups: ['Coil KMH1']
  Status events: 3
    Coil K

## How Daily Performance Counts Trucks

Daily Performance aggregates by Product_Group and Coming_to_Load_or_Unload, counting distinct trucks

In [12]:
# Simulate Daily Performance aggregation
print("=== DAILY PERFORMANCE SIMULATION ===\n")

# Add Coming_to_Load_or_Unload to KPI (matching the component logic)
df_security_dated = dfs['security'].copy()
df_security_dated["Timestamp"] = pd.to_datetime(df_security_dated["Timestamp"], errors="coerce")
df_security_dated["_Date"] = df_security_dated["Timestamp"].dt.date

# Filter to today
sec_today = df_security_dated[df_security_dated["_Date"] == today]

if not sec_today.empty:
    sec_map = (
        sec_today.sort_values("Timestamp")
        .groupby(["Truck_Plate_Number", "_Date"])["Coming_to_Load_or_Unload"]
        .first()
        .reset_index()
        .rename(columns={"_Date": "Date"})
    )
    
    df_kpi_with_upload = df_kpi.merge(sec_map, on=["Truck_Plate_Number", "Date"], how="left")
    df_kpi_with_upload["Coming_to_Load_or_Unload"] = df_kpi_with_upload["Coming_to_Load_or_Unload"].fillna("⚠️ NO SECURITY RECORD")
else:
    df_kpi_with_upload = df_kpi.copy()
    df_kpi_with_upload["Coming_to_Load_or_Unload"] = "⚠️ NO SECURITY RECORD"

# Now filter by Unloading (if upload_type was specified)
coil_unloading = df_kpi_with_upload[df_kpi_with_upload["Coming_to_Load_or_Unload"] == "Unloading"]

print(f"KPI rows after Coming_to_Load_or_Unload filter: {len(coil_unloading)}")
print(f"Unique trucks: {coil_unloading['Truck_Plate_Number'].nunique()}")

# This is what Daily Performance displays - count of unique trucks per (Product, Upload Type)
agg = coil_unloading.groupby(["Product_Group", "Coming_to_Load_or_Unload"]).agg(
    Total_truck=("Truck_Plate_Number", "nunique")
).reset_index()

print("\n=== AGGREGATED FOR DAILY PERFORMANCE ===")
print(agg.to_string(index=False))

# Show the truck list
print(f"\n=== TRUCK LIST IN DAILY PERFORMANCE ===")
print(f"Trucks counted: {sorted(coil_unloading['Truck_Plate_Number'].unique())}")

=== DAILY PERFORMANCE SIMULATION ===

KPI rows after Coming_to_Load_or_Unload filter: 9
Unique trucks: 9

=== AGGREGATED FOR DAILY PERFORMANCE ===
Product_Group Coming_to_Load_or_Unload  Total_truck
    Coil KMH1                Unloading            9

=== TRUCK LIST IN DAILY PERFORMANCE ===
Trucks counted: ['3A-2389', '3A-2389-2', '3A-2389-3', '3A-6126', '3A-6355', '3A-6355-3', '3B-5677', '3B-5677-2', 'TK-0100']


## Detailed Analysis: What's in Loading Durations Status?

Check each row to see if trucks appear multiple times

In [ ]:
# Loading Durations Status shows one row per (Truck, Product, Date) but also filters by Coming_to_Load_or_Unload
print("=== LOADING DURATIONS STATUS ROWS ===\n")

# The component applies the upload_type filter on df_kpi (which already has Product filter)
# Then adds Coming_to_Load_or_Unload from security
# Let's replicate that

# Start with all Coil KMH1 from KPI (already filtered by product)
print(f"Step 1 - KPI with product filter (Coil KMH1): {len(df_kpi)} rows")
print(f"  Trucks in KPI: {sorted(df_kpi['Truck_Plate_Number'].unique())}")

# Add Coming_to_Load_or_Unload (using date range logic)
sec_filtered = sec_today.copy()
if not sec_filtered.empty:
    coming_map = (
        sec_filtered.sort_values("Timestamp")
        .groupby("Truck_Plate_Number")["Coming_to_Load_or_Unload"]
        .first()
        .reset_index()
    )
    df_durations = df_kpi.merge(coming_map, on="Truck_Plate_Number", how="left")
    df_durations["Coming_to_Load_or_Unload"] = df_durations["Coming_to_Load_or_Unload"].fillna("⚠️ NO SECURITY RECORD")
else:
    df_durations = df_kpi.copy()
    df_durations["Coming_to_Load_or_Unload"] = "⚠️ NO SECURITY RECORD"

print(f"\nStep 2 - After adding Coming_to_Load_or_Unload: {len(df_durations)} rows")

# Count by Coming_to_Load_or_Unload
print("\nBreakdown by Coming_to_Load_or_Unload:")
print(df_durations["Coming_to_Load_or_Unload"].value_counts())

# Filter to Unloading only (simulating the upload_type filter)
df_unloading_only = df_durations[df_durations["Coming_to_Load_or_Unload"] == "Unloading"]
print(f"\nStep 3 - After filtering to Unloading only: {len(df_unloading_only)} rows")
print(f"  Unique trucks: {df_unloading_only['Truck_Plate_Number'].nunique()}")

# Show all rows
print("\n=== ALL 9 ROWS IN LOADING DURATIONS STATUS ===")
print(df_unloading_only[["Truck_Plate_Number", "Product_Group", "Coming_to_Load_or_Unload"]].to_string(index=False))

# Check for duplicates
print("\n=== TRUCK APPEARANCE COUNT ===")
truck_counts = df_unloading_only["Truck_Plate_Number"].value_counts().sort_index()
print(truck_counts)

## Replicate Daily Performance's Complex Matching Logic

Daily Performance uses timestamp matching within 60min window instead of simple merge

# Investigation: Roofing Coming_to_Load_or_Unload Discrepancy

Checking why Loading Durations Status shows "Loading" and "None" but Daily Performance shows "Loading" and "Unloading" for Roofing

In [6]:
import pandas as pd
from config.config import SPREADSHEET_ID, SHEET_GIDS
from data.processor import clean_sheet_dfs
from data.metrics import compute_per_truck_metrics

# Load and clean data
def _sheet_csv_url(gid: str):
    return f"https://docs.google.com/spreadsheets/d/{SPREADSHEET_ID}/export?format=csv&gid={gid}"

raw_dfs = {}
for sheet_name in ['security', 'driver', 'status', 'logistic']:
    url = _sheet_csv_url(SHEET_GIDS[sheet_name])
    raw_dfs[sheet_name] = pd.read_csv(url, dtype=str, keep_default_na=False)

dfs = clean_sheet_dfs(raw_dfs)

# Get today's date
today = pd.Timestamp.now(tz="Asia/Phnom_Penh").date()
print(f"Analyzing data for: {today}\n")

# Compute KPI
df_kpi = compute_per_truck_metrics(
    dfs['security'], dfs['status'], dfs['logistic'], dfs['driver'],
    start_date=today,
    end_date=today,
    product_filter=None,
    upload_type=None,
    use_fallbacks=False
)

# Filter for Roofing
roofing_kpi = df_kpi[df_kpi["Product_Group"] == "Roofing"].copy()
print(f"Total Roofing KPI rows: {len(roofing_kpi)}")
print(f"Roofing trucks: {sorted(roofing_kpi['Truck_Plate_Number'].unique())}")

Analyzing data for: 2025-12-23

Total Roofing KPI rows: 8
Roofing trucks: ['1JO-2735', '2AD-9220', '3A-2669', '3A-2669-2', '3E-5953-2', '3F-5849-2', '3F-6447-2', 'PSHN-0585']


## Method 1: Loading Durations Status Logic
Uses latest Coming_to_Load_or_Unload per truck (across ALL dates)

In [7]:
# Simulate Loading Durations Status logic
df_security = dfs['security'].copy()
df_security["Timestamp"] = pd.to_datetime(df_security["Timestamp"], errors="coerce")

print("=== LOADING DURATIONS STATUS METHOD ===")
print("Uses: Latest Coming_to_Load_or_Unload per truck (historical)\n")

if "Coming_to_Load_or_Unload" in df_security.columns and "Truck_Plate_Number" in df_security.columns:
    # This is what loading_durations_status.py does
    coming_map = (
        df_security.sort_values("Timestamp")
        .groupby("Truck_Plate_Number")["Coming_to_Load_or_Unload"]
        .last()  # Latest across ALL dates
        .reset_index()
    )
    
    roofing_method1 = roofing_kpi.merge(coming_map, on="Truck_Plate_Number", how="left")
    roofing_method1["Coming_to_Load_or_Unload"] = roofing_method1["Coming_to_Load_or_Unload"].fillna("None")
    
    print("Roofing trucks with Coming_to_Load_or_Unload:")
    result1 = roofing_method1[["Truck_Plate_Number", "Coming_to_Load_or_Unload"]].drop_duplicates().sort_values("Truck_Plate_Number")
    print(result1.to_string(index=False))
    
    print("\n\nBreakdown:")
    breakdown1 = roofing_method1["Coming_to_Load_or_Unload"].value_counts()
    print(breakdown1)
    
    # Check which trucks have NO security records at all
    roofing_trucks = roofing_kpi["Truck_Plate_Number"].unique()
    trucks_with_security = coming_map["Truck_Plate_Number"].unique()
    trucks_without_security = [t for t in roofing_trucks if t not in trucks_with_security]
    
    if trucks_without_security:
        print(f"\n\nTrucks with NO security records (showing as None): {trucks_without_security}")

=== LOADING DURATIONS STATUS METHOD ===
Uses: Latest Coming_to_Load_or_Unload per truck (historical)

Roofing trucks with Coming_to_Load_or_Unload:
Truck_Plate_Number Coming_to_Load_or_Unload
          1JO-2735                     None
          2AD-9220                  Loading
           3A-2669                  Loading
         3A-2669-2                  Loading
         3E-5953-2                  Loading
         3F-5849-2                  Loading
         3F-6447-2                  Loading
         PSHN-0585                  Loading


Breakdown:
Coming_to_Load_or_Unload
Loading    7
None       1
Name: count, dtype: int64


Trucks with NO security records (showing as None): ['1JO-2735']


## Method 2: Daily Performance Logic
Uses Coming_to_Load_or_Unload per truck + date (today's records)

In [8]:
# Simulate Daily Performance logic
df_security_dated = dfs['security'].copy()
df_security_dated["Timestamp"] = pd.to_datetime(df_security_dated["Timestamp"], errors="coerce")
df_security_dated["_Date"] = df_security_dated["Timestamp"].dt.date

print("=== DAILY PERFORMANCE METHOD ===")
print("Uses: First Coming_to_Load_or_Unload per truck + date (today only)\n")

if "Truck_Plate_Number" in df_security_dated.columns:
    # This is what daily_performance.py does
    sec_map = (
        df_security_dated.sort_values("Timestamp")
        .groupby(["Truck_Plate_Number", "_Date"])["Coming_to_Load_or_Unload"]
        .first()  # First scan on that date
        .reset_index()
        .rename(columns={"_Date": "Date"})
    )
    
    roofing_method2 = roofing_kpi.merge(sec_map, on=["Truck_Plate_Number", "Date"], how="left")
    roofing_method2["Coming_to_Load_or_Unload_Display"] = roofing_method2["Coming_to_Load_or_Unload"].fillna("⚠️ NO SECURITY RECORD")
    
    print("Roofing trucks with Coming_to_Load_or_Unload (by date):")
    result2 = roofing_method2[["Truck_Plate_Number", "Date", "Coming_to_Load_or_Unload_Display"]].sort_values("Truck_Plate_Number")
    print(result2.to_string(index=False))
    
    print("\n\nBreakdown:")
    breakdown2 = roofing_method2["Coming_to_Load_or_Unload_Display"].value_counts()
    print(breakdown2)
    
    # Check which trucks have security records today
    sec_today = df_security_dated[df_security_dated["_Date"] == today]
    roofing_trucks = roofing_kpi["Truck_Plate_Number"].unique()
    trucks_with_sec_today = sec_today["Truck_Plate_Number"].unique()
    trucks_without_sec_today = [t for t in roofing_trucks if t not in trucks_with_sec_today]
    
    if trucks_without_sec_today:
        print(f"\n\nTrucks with NO security records TODAY: {trucks_without_sec_today}")

=== DAILY PERFORMANCE METHOD ===
Uses: First Coming_to_Load_or_Unload per truck + date (today only)

Roofing trucks with Coming_to_Load_or_Unload (by date):
Truck_Plate_Number       Date Coming_to_Load_or_Unload_Display
          1JO-2735 2025-12-23            ⚠️ NO SECURITY RECORD
          2AD-9220 2025-12-23                          Loading
           3A-2669 2025-12-23                          Loading
         3A-2669-2 2025-12-23                          Loading
         3E-5953-2 2025-12-23                          Loading
         3F-5849-2 2025-12-23                          Loading
         3F-6447-2 2025-12-23                          Loading
         PSHN-0585 2025-12-23            ⚠️ NO SECURITY RECORD


Breakdown:
Coming_to_Load_or_Unload_Display
Loading                  6
⚠️ NO SECURITY RECORD    2
Name: count, dtype: int64


Trucks with NO security records TODAY: ['1JO-2735', 'PSHN-0585']


## Detailed Check: Security Records for Each Roofing Truck

In [9]:
# Check each Roofing truck's security records in detail
roofing_trucks = sorted(roofing_kpi["Truck_Plate_Number"].unique())

print("=== DETAILED SECURITY RECORDS FOR EACH ROOFING TRUCK ===\n")

for truck in roofing_trucks:
    print(f"\n{'='*60}")
    print(f"Truck: {truck}")
    print(f"{'='*60}")
    
    # All historical security records
    all_sec = df_security[df_security["Truck_Plate_Number"] == truck].sort_values("Timestamp")
    
    if all_sec.empty:
        print("  ❌ NO SECURITY RECORDS (EVER)")
        continue
    
    # Today's security records
    today_sec = all_sec[all_sec["Timestamp"].dt.date == today]
    
    print(f"\n  Historical records: {len(all_sec)}")
    print(f"  Latest historical record:")
    latest = all_sec.iloc[-1]
    print(f"    Date: {latest['Timestamp']}")
    print(f"    Coming_to_Load_or_Unload: {latest['Coming_to_Load_or_Unload']}")
    
    print(f"\n  Today's records: {len(today_sec)}")
    if not today_sec.empty:
        print(f"  First scan today:")
        first_today = today_sec.iloc[0]
        print(f"    Time: {first_today['Timestamp']}")
        print(f"    Coming_to_Load_or_Unload: {first_today['Coming_to_Load_or_Unload']}")
        
        if len(today_sec) > 1:
            print(f"  All scans today:")
            for _, row in today_sec.iterrows():
                print(f"    {row['Timestamp'].strftime('%H:%M:%S')} - {row['Coming_to_Load_or_Unload']}")
    else:
        print("  ⚠️  NO SECURITY RECORDS TODAY")
        print(f"  → Loading Durations Status will use: {latest['Coming_to_Load_or_Unload']} (historical)")
        print(f"  → Daily Performance will show: NO SECURITY RECORD")

=== DETAILED SECURITY RECORDS FOR EACH ROOFING TRUCK ===


Truck: 1JO-2735
  ❌ NO SECURITY RECORDS (EVER)

Truck: 2AD-9220

  Historical records: 33
  Latest historical record:
    Date: 2025-12-23 15:57:32+07:00
    Coming_to_Load_or_Unload: Loading

  Today's records: 1
  First scan today:
    Time: 2025-12-23 15:57:32+07:00
    Coming_to_Load_or_Unload: Loading

Truck: 3A-2669

  Historical records: 72
  Latest historical record:
    Date: 2025-12-23 11:56:28+07:00
    Coming_to_Load_or_Unload: Loading

  Today's records: 1
  First scan today:
    Time: 2025-12-23 11:56:28+07:00
    Coming_to_Load_or_Unload: Loading

Truck: 3A-2669-2

  Historical records: 23
  Latest historical record:
    Date: 2025-12-23 15:32:11+07:00
    Coming_to_Load_or_Unload: Loading

  Today's records: 2
  First scan today:
    Time: 2025-12-23 14:13:05+07:00
    Coming_to_Load_or_Unload: Loading
  All scans today:
    14:13:05 - Loading
    15:32:11 - Loading

Truck: 3E-5953-2

  Historical records: 37
  

## Summary & Explanation

**Why the discrepancy?**

1. **Loading Durations Status** uses the **latest historical** Coming_to_Load_or_Unload for each truck (across all dates)
   - If a truck has no security record at all → shows "None"
   - If a truck visited last week for "Unloading" but has no security scan today → shows "Unloading"

2. **Daily Performance** uses Coming_to_Load_or_Unload from **today's security records only**
   - If a truck has no security scan today → shows "NO SECURITY RECORD"
   - Only shows what was scanned at the gate TODAY

**The difference:** Loading Durations Status falls back to historical data, while Daily Performance only looks at today.

In [ ]:
#!/usr/bin/env python3
"""Quick test to see what columns are in the raw sheets."""
import pandas as pd
from config.config import SPREADSHEET_ID, SHEET_GIDS

def _sheet_csv_url(gid: str):
    return f"https://docs.google.com/spreadsheets/d/{SPREADSHEET_ID}/export?format=csv&gid={gid}"

print("\n=== RAW COLUMNS FROM SHEETS ===\n")

for sheet_name in ['security', 'driver', 'status', 'logistic']:
    url = _sheet_csv_url(SHEET_GIDS[sheet_name])
    df = pd.read_csv(url, dtype=str, keep_default_na=False, nrows=5)
    print(f"\n{sheet_name.upper()} sheet:")
    print(f"  Columns: {list(df.columns)}")
    print(f"  First row sample: {df.iloc[0].to_dict() if len(df) > 0 else 'EMPTY'}")


# Check Security Data - Loading vs Unloading

Let's investigate the Coming_to_Load_or_Unload field in security sheet

In [1]:
import pandas as pd
from config.config import SPREADSHEET_ID, SHEET_GIDS
from data.processor import clean_sheet_dfs

# Load raw sheets
def _sheet_csv_url(gid: str):
    return f"https://docs.google.com/spreadsheets/d/{SPREADSHEET_ID}/export?format=csv&gid={gid}"

raw_dfs = {}
for sheet_name in ['security', 'driver', 'status', 'logistic']:
    url = _sheet_csv_url(SHEET_GIDS[sheet_name])
    raw_dfs[sheet_name] = pd.read_csv(url, dtype=str, keep_default_na=False)

# Clean the data
dfs = clean_sheet_dfs(raw_dfs)

# Check today's data
today = pd.Timestamp.now(tz="Asia/Phnom_Penh").date()
print(f"Checking data for: {today}\n")

# Security sheet - Coming_to_Load_or_Unload
df_security = dfs['security'].copy()
df_security["Timestamp"] = pd.to_datetime(df_security["Timestamp"], errors="coerce")
df_security["Date"] = df_security["Timestamp"].dt.date

security_today = df_security[df_security["Date"] == today]

print("=== SECURITY DATA TODAY ===")
print(f"Total records: {len(security_today)}")
if "Coming_to_Load_or_Unload" in security_today.columns:
    print("\nComing_to_Load_or_Unload breakdown:")
    print(security_today["Coming_to_Load_or_Unload"].value_counts())
    print("\nUnique values:")
    print(security_today["Coming_to_Load_or_Unload"].unique())
else:
    print("Coming_to_Load_or_Unload column not found!")

print("\n=== STATUS DATA TODAY (for Pipe) ===")
df_status = dfs['status'].copy()
df_status["Timestamp"] = pd.to_datetime(df_status["Timestamp"], errors="coerce")
df_status["Date"] = df_status["Timestamp"].dt.date

status_today = df_status[df_status["Date"] == today]
if "Product_Group" in status_today.columns:
    status_pipe = status_today[status_today["Product_Group"] == "Pipe"]
    print(f"Pipe records: {len(status_pipe)}")
    print("\nTruck plates with Pipe today:")
    print(status_pipe["Truck_Plate_Number"].unique())
    
    # Check what security says about these trucks
    if len(status_pipe) > 0:
        pipe_trucks = status_pipe["Truck_Plate_Number"].unique()
        security_pipe = security_today[security_today["Truck_Plate_Number"].isin(pipe_trucks)]
        print(f"\nSecurity records for Pipe trucks:")
        print(security_pipe[["Truck_Plate_Number", "Coming_to_Load_or_Unload"]].drop_duplicates())

Checking data for: 2025-12-23

=== SECURITY DATA TODAY ===
Total records: 102

Coming_to_Load_or_Unload breakdown:
Coming_to_Load_or_Unload
Loading      81
Unloading    21
Name: count, dtype: int64

Unique values:
['Loading' 'Unloading']

=== STATUS DATA TODAY (for Pipe) ===
Pipe records: 28

Truck plates with Pipe today:
['3B-0944' '3E-7207' '3A-1138' '3G-0836' '3G-0837' '3B-5956-2' '3A-2669-2'
 '3C-8001-2' '3D-1664' '3A-6683' '3C-8001' '3A-3559' '3E-2588-2' '3G-8316'
 '3E-6115-2' '2BI-9735' '3G-7863' '3G-8278' 'PSHN-0585']

Security records for Pipe trucks:
     Truck_Plate_Number Coming_to_Load_or_Unload
4843            3E-7207                  Loading
4844            3B-0944                  Loading
4845            3A-1138                  Loading
4851          3B-5956-2                  Loading
4866            3G-0836                  Loading
4867            3G-0837                  Loading
4869            3A-6683                  Loading
4871            3G-7863                  L

# Check Roofing Data - Why 5 in Loading Durations but 4+1(None) in Daily Performance?

In [2]:
# Check the 5 Roofing trucks from today
roofing_trucks = ['3A-2669', '3E-5953-2', '3F-5849-2', '3F-6447-2', 'PSHN-0585']

print("=== ROOFING TRUCKS - SECURITY DATA ===")
for truck in roofing_trucks:
    sec_truck = security_today[security_today["Truck_Plate_Number"] == truck]
    if not sec_truck.empty:
        print(f"\n{truck}:")
        print(f"  Coming_to_Load_or_Unload: {sec_truck['Coming_to_Load_or_Unload'].unique()}")
        print(f"  First timestamp: {sec_truck['Timestamp'].min()}")
    else:
        print(f"\n{truck}: *** NO SECURITY RECORD ***")

print("\n\n=== ROOFING TRUCKS - STATUS DATA ===")
for truck in roofing_trucks:
    stat_truck = status_today[status_today["Truck_Plate_Number"] == truck]
    if not stat_truck.empty:
        print(f"\n{truck}:")
        print(f"  Product_Group: {stat_truck['Product_Group'].unique()}")
        print(f"  Status events: {stat_truck['Status'].tolist()}")
        print(f"  Timestamps: {stat_truck['Timestamp'].tolist()}")
    else:
        print(f"\n{truck}: *** NO STATUS RECORD ***")

# Now check what daily_performance would compute
print("\n\n=== CHECKING KPI COMPUTATION ===")
from data.metrics import compute_per_truck_metrics

df_kpi = compute_per_truck_metrics(
    dfs['security'], dfs['status'], dfs['logistic'], dfs['driver'],
    start_date=today,
    end_date=today,
    product_filter=None,
    upload_type=None,
    use_fallbacks=False
)

print(f"\nTotal KPI rows: {len(df_kpi)}")
roofing_kpi = df_kpi[df_kpi["Product_Group"] == "Roofing"]
print(f"Roofing KPI rows: {len(roofing_kpi)}")
print("\nRoofing trucks in KPI:")
print(roofing_kpi[["Truck_Plate_Number", "Product_Group", "Date"]].to_string())

# Now add Coming_to_Load_or_Unload the way daily_performance does
df_security_copy = dfs['security'].copy()
df_security_copy["Timestamp"] = pd.to_datetime(df_security_copy["Timestamp"], errors="coerce")
df_security_copy["_Date"] = df_security_copy["Timestamp"].dt.date

if "Truck_Plate_Number" in df_security_copy.columns:
    security_map = (
        df_security_copy.sort_values("Timestamp")
        .groupby(["Truck_Plate_Number", "_Date"])["Coming_to_Load_or_Unload"]
        .first()
        .reset_index()
        .rename(columns={"_Date": "Date"})
    )
    
    roofing_kpi = roofing_kpi.merge(
        security_map,
        on=["Truck_Plate_Number", "Date"],
        how="left"
    )

print("\n\n=== ROOFING KPI WITH Coming_to_Load_or_Unload ===")
print(roofing_kpi[["Truck_Plate_Number", "Product_Group", "Coming_to_Load_or_Unload"]].to_string())

=== ROOFING TRUCKS - SECURITY DATA ===

3A-2669:
  Coming_to_Load_or_Unload: ['Loading']
  First timestamp: 2025-12-23 11:56:28+07:00

3E-5953-2:
  Coming_to_Load_or_Unload: ['Loading']
  First timestamp: 2025-12-23 10:53:38+07:00

3F-5849-2:
  Coming_to_Load_or_Unload: ['Loading']
  First timestamp: 2025-12-23 13:11:23+07:00

3F-6447-2:
  Coming_to_Load_or_Unload: ['Loading']
  First timestamp: 2025-12-23 08:27:45+07:00

PSHN-0585: *** NO SECURITY RECORD ***


=== ROOFING TRUCKS - STATUS DATA ===

3A-2669: *** NO STATUS RECORD ***

3E-5953-2:
  Product_Group: ['Roofing']
  Status events: ['Start_Loading', 'Completed']
  Timestamps: [Timestamp('2025-12-23 12:14:40+0700', tz='Asia/Phnom_Penh'), Timestamp('2025-12-23 12:55:47+0700', tz='Asia/Phnom_Penh')]

3F-5849-2:
  Product_Group: ['CZD' 'Roofing']
  Status events: ['Start_Loading', 'Completed', 'Start_Loading', 'Completed']
  Timestamps: [Timestamp('2025-12-23 13:35:20+0700', tz='Asia/Phnom_Penh'), Timestamp('2025-12-23 13:56:58+0700

In [3]:
# Check PSHN-0585 across ALL security records (not just today)
print("\n=== PSHN-0585 - ALL SECURITY RECORDS ===")
df_security_all = dfs['security'].copy()
pshn_sec = df_security_all[df_security_all["Truck_Plate_Number"] == "PSHN-0585"]
if not pshn_sec.empty:
    print(f"Found {len(pshn_sec)} security records for PSHN-0585")
    print(pshn_sec[["Timestamp", "Coming_to_Load_or_Unload"]].tail(10))
    print(f"\nLatest Coming_to_Load_or_Unload: {pshn_sec.sort_values('Timestamp')['Coming_to_Load_or_Unload'].iloc[-1]}")
else:
    print("NO security records found for PSHN-0585 (even historically)")

# Check what loading_durations_status would show
print("\n=== LOADING DURATIONS STATUS MERGE LOGIC ===")
if "Coming_to_Load_or_Unload" in df_security_all.columns:
    coming_map = (
        df_security_all.sort_values("Timestamp")
        .groupby("Truck_Plate_Number")["Coming_to_Load_or_Unload"]
        .last()  # Uses LAST, not first
        .reset_index()
    )
    pshn_coming = coming_map[coming_map["Truck_Plate_Number"] == "PSHN-0585"]
    if not pshn_coming.empty:
        print(f"PSHN-0585 Coming_to_Load_or_Unload from historical: {pshn_coming['Coming_to_Load_or_Unload'].iloc[0]}")
    else:
        print("PSHN-0585 not in coming_map (no security records)")


=== PSHN-0585 - ALL SECURITY RECORDS ===
Found 1 security records for PSHN-0585
                     Timestamp Coming_to_Load_or_Unload
3905 2025-12-15 21:40:00+07:00                  Loading

Latest Coming_to_Load_or_Unload: Loading

=== LOADING DURATIONS STATUS MERGE LOGIC ===
PSHN-0585 Coming_to_Load_or_Unload from historical: Loading


# Check PU Data - Why 1 Loading and 4 None?

In [4]:
# Get PU trucks from today's status
pu_status = status_today[status_today["Product_Group"] == "PU"]
pu_trucks = pu_status["Truck_Plate_Number"].unique()

print("=== PU TRUCKS TODAY ===")
print(f"Found {len(pu_trucks)} PU trucks: {list(pu_trucks)}")

print("\n=== PU TRUCKS - SECURITY DATA TODAY ===")
for truck in pu_trucks:
    sec_truck = security_today[security_today["Truck_Plate_Number"] == truck]
    if not sec_truck.empty:
        print(f"\n{truck}:")
        print(f"  Coming_to_Load_or_Unload: {sec_truck['Coming_to_Load_or_Unload'].unique()}")
        print(f"  First timestamp: {sec_truck['Timestamp'].min()}")
    else:
        print(f"\n{truck}: *** NO SECURITY RECORD TODAY ***")

print("\n\n=== PU TRUCKS - HISTORICAL SECURITY DATA ===")
for truck in pu_trucks:
    sec_truck_all = df_security_all[df_security_all["Truck_Plate_Number"] == truck]
    if not sec_truck_all.empty:
        latest = sec_truck_all.sort_values("Timestamp").iloc[-1]
        print(f"\n{truck}:")
        print(f"  Total historical records: {len(sec_truck_all)}")
        print(f"  Latest record date: {latest['Timestamp']}")
        print(f"  Latest Coming_to_Load_or_Unload: {latest['Coming_to_Load_or_Unload']}")
    else:
        print(f"\n{truck}: *** NO HISTORICAL SECURITY RECORD ***")

# Check what the aggregation would show
print("\n\n=== PU KPI AGGREGATION ===")
pu_kpi = df_kpi[df_kpi["Product_Group"] == "PU"]
print(f"Total PU KPI rows: {len(pu_kpi)}")

# Add Coming_to_Load_or_Unload with the NEW logic (date-based + historical fallback)
if "Truck_Plate_Number" in df_security_copy.columns:
    # Strategy 1: Try to match by Truck + Date (preferred)
    security_map_by_date = (
        df_security_copy.sort_values("Timestamp")
        .groupby(["Truck_Plate_Number", "_Date"])["Coming_to_Load_or_Unload"]
        .first()
        .reset_index()
        .rename(columns={"_Date": "Date"})
    )
    
    # Strategy 2: Fallback to last historical record per truck
    security_map_historical = (
        df_security_copy.sort_values("Timestamp")
        .groupby("Truck_Plate_Number")["Coming_to_Load_or_Unload"]
        .last()
        .reset_index()
        .rename(columns={"Coming_to_Load_or_Unload": "Coming_to_Load_or_Unload_Historical"})
    )
    
    # Merge by Truck + Date first
    pu_kpi = pu_kpi.merge(
        security_map_by_date,
        on=["Truck_Plate_Number", "Date"],
        how="left"
    )
    
    # Then merge historical as fallback
    pu_kpi = pu_kpi.merge(
        security_map_historical,
        on="Truck_Plate_Number",
        how="left"
    )
    
    # Use historical if today's is missing
    if "Coming_to_Load_or_Unload" not in pu_kpi.columns:
        pu_kpi["Coming_to_Load_or_Unload_Final"] = pu_kpi["Coming_to_Load_or_Unload_Historical"]
    else:
        pu_kpi["Coming_to_Load_or_Unload_Final"] = pu_kpi["Coming_to_Load_or_Unload"].fillna(
            pu_kpi["Coming_to_Load_or_Unload_Historical"]
        )

print("\nPU trucks with Coming_to_Load_or_Unload values:")
print(pu_kpi[["Truck_Plate_Number", "Product_Group", "Coming_to_Load_or_Unload", "Coming_to_Load_or_Unload_Historical", "Coming_to_Load_or_Unload_Final"]].to_string())

print("\n\n=== AGGREGATION BY Coming_to_Load_or_Unload ===")
agg = pu_kpi.groupby("Coming_to_Load_or_Unload_Final").size()
print(agg)

=== PU TRUCKS TODAY ===
Found 5 PU trucks: ['1Z-7839', '1CE-5512', '1L-5035', '1IG-9159-3', '3B-8278']

=== PU TRUCKS - SECURITY DATA TODAY ===

1Z-7839: *** NO SECURITY RECORD TODAY ***

1CE-5512: *** NO SECURITY RECORD TODAY ***

1L-5035: *** NO SECURITY RECORD TODAY ***

1IG-9159-3: *** NO SECURITY RECORD TODAY ***

3B-8278:
  Coming_to_Load_or_Unload: ['Loading']
  First timestamp: 2025-12-23 12:56:48+07:00


=== PU TRUCKS - HISTORICAL SECURITY DATA ===

1Z-7839: *** NO HISTORICAL SECURITY RECORD ***

1CE-5512: *** NO HISTORICAL SECURITY RECORD ***

1L-5035: *** NO HISTORICAL SECURITY RECORD ***

1IG-9159-3: *** NO HISTORICAL SECURITY RECORD ***

3B-8278:
  Total historical records: 27
  Latest record date: 2025-12-23 12:56:48+07:00
  Latest Coming_to_Load_or_Unload: Loading


=== PU KPI AGGREGATION ===
Total PU KPI rows: 5

PU trucks with Coming_to_Load_or_Unload values:
  Truck_Plate_Number Product_Group Coming_to_Load_or_Unload Coming_to_Load_or_Unload_Historical Coming_to_Load_

# Check Trading Data - Why 4 vs 10 NO SECURITY RECORD?

In [5]:
# Get Trading trucks from today's status
trading_status = status_today[status_today["Product_Group"] == "Trading"]
trading_trucks = trading_status["Truck_Plate_Number"].unique()

print("=== TRADING TRUCKS TODAY ===")
print(f"Found {len(trading_trucks)} Trading trucks total")
print(f"Trucks: {sorted(trading_trucks)}")

# Check which ones have security records
print("\n=== SECURITY RECORDS CHECK ===")
no_security = []
has_security = []

for truck in trading_trucks:
    sec_truck = security_today[security_today["Truck_Plate_Number"] == truck]
    if sec_truck.empty:
        no_security.append(truck)
    else:
        has_security.append(truck)

print(f"\nTrucks WITH security records: {len(has_security)}")
print(f"  {sorted(has_security)}")
print(f"\nTrucks WITHOUT security records: {len(no_security)}")
print(f"  {sorted(no_security)}")

# Now check what Loading Durations Status would show (uses df_kpi from compute_per_truck_metrics)
print("\n\n=== LOADING DURATIONS STATUS SIMULATION ===")
trading_kpi = df_kpi[df_kpi["Product_Group"] == "Trading"].copy()
print(f"Total Trading KPI rows: {len(trading_kpi)}")
print(f"Truck plates in KPI: {sorted(trading_kpi['Truck_Plate_Number'].unique())}")

# Add Coming_to_Load_or_Unload like loading_durations_status does (by truck only, all dates)
if "Coming_to_Load_or_Unload" in df_security.columns:
    coming_map_durations = (
        df_security.sort_values("Timestamp")
        .groupby("Truck_Plate_Number")["Coming_to_Load_or_Unload"]
        .last()
        .reset_index()
    )
    trading_kpi = trading_kpi.merge(coming_map_durations, on="Truck_Plate_Number", how="left")
    trading_kpi["Coming_to_Load_or_Unload"] = trading_kpi["Coming_to_Load_or_Unload"].fillna("⚠️ NO SECURITY RECORD")

print("\nTrading KPI with Coming_to_Load_or_Unload:")
print(trading_kpi[["Truck_Plate_Number", "Coming_to_Load_or_Unload"]].drop_duplicates().to_string())

no_sec_count_durations = len(trading_kpi[trading_kpi["Coming_to_Load_or_Unload"] == "⚠️ NO SECURITY RECORD"])
print(f"\nNO SECURITY RECORD count in Loading Durations Status: {no_sec_count_durations}")

# Now check what Daily Performance would show (by truck + date)
print("\n\n=== DAILY PERFORMANCE SIMULATION ===")
trading_kpi2 = df_kpi[df_kpi["Product_Group"] == "Trading"].copy()

# Add Coming_to_Load_or_Unload like daily_performance does (by truck + date)
if "Truck_Plate_Number" in df_security_copy.columns:
    security_map_by_date = (
        df_security_copy.sort_values("Timestamp")
        .groupby(["Truck_Plate_Number", "_Date"])["Coming_to_Load_or_Unload"]
        .first()
        .reset_index()
        .rename(columns={"_Date": "Date"})
    )
    
    trading_kpi2 = trading_kpi2.merge(
        security_map_by_date,
        on=["Truck_Plate_Number", "Date"],
        how="left"
    )
    trading_kpi2["Coming_to_Load_or_Unload_Display"] = trading_kpi2["Coming_to_Load_or_Unload"].fillna("⚠️ NO SECURITY RECORD")

print("Trading KPI with Coming_to_Load_or_Unload (by date):")
print(trading_kpi2[["Truck_Plate_Number", "Date", "Coming_to_Load_or_Unload_Display"]].to_string())

# Aggregate like daily_performance does
agg_trading = trading_kpi2.groupby("Coming_to_Load_or_Unload_Display").size()
print("\nAggregation for Daily Performance:")
print(agg_trading)

print(f"\nNO SECURITY RECORD count in Daily Performance: {agg_trading.get('⚠️ NO SECURITY RECORD', 0)}")

=== TRADING TRUCKS TODAY ===
Found 16 Trading trucks total
Trucks: ['3A-1607', '3A-2395-1', '3C-2257', '3E-1042-1', '3E-6447', '3E-6548', '3F-3122', '3F-4708', '3F-6393', '3F-6447', '3F-7926', '3F-8691', '3F-8691-1', '3G-0837', '3G-7861', '3G-7863']

=== SECURITY RECORDS CHECK ===

Trucks WITH security records: 6
  ['3A-1607', '3E-6548', '3F-8691', '3G-0837', '3G-7861', '3G-7863']

Trucks WITHOUT security records: 10
  ['3A-2395-1', '3C-2257', '3E-1042-1', '3E-6447', '3F-3122', '3F-4708', '3F-6393', '3F-6447', '3F-7926', '3F-8691-1']


=== LOADING DURATIONS STATUS SIMULATION ===
Total Trading KPI rows: 16
Truck plates in KPI: ['3A-1607', '3A-2395-1', '3C-2257', '3E-1042-1', '3E-6447', '3E-6548', '3F-3122', '3F-4708', '3F-6393', '3F-6447', '3F-7926', '3F-8691', '3F-8691-1', '3G-0837', '3G-7861', '3G-7863']

Trading KPI with Coming_to_Load_or_Unload:
   Truck_Plate_Number Coming_to_Load_or_Unload
0             3A-1607                  Loading
1           3A-2395-1    ⚠️ NO SECURITY RECOR